In [1]:
import numpy as np
import pandas as pd

In [8]:
df1 = pd.read_csv('2016.csv')
print(df1.shape)
df1.head()

(3141, 11)


,Unnamed: 0,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips
0,0,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2013
1,1,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2016
2,2,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2020
3,3,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2050
4,4,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2060


In [9]:
df2 = pd.read_csv('2020.csv')
print(df2.shape)
df2.head()

(3159, 10)


,state_name,county_fips,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff
0,Alabama,1001,Autauga County,19764,7450,27639,12314,0.715077,0.269547,0.445530
1,Alabama,1003,Baldwin County,83055,24344,108945,58711,0.762357,0.223452,0.538905
2,Alabama,1005,Barbour County,5605,4772,10457,833,0.536005,0.456345,0.079660
3,Alabama,1007,Bibb County,7508,1982,9573,5526,0.784289,0.207041,0.577249
4,Alabama,1009,Blount County,24595,2627,27459,21968,0.895699,0.095670,0.800029


In [ ]:
'''
To-do:
- Determine why # of rows don't match
- Create new column with class ID (0 if GOP won, 1 if DEM won)
- Pull in data from other sources + join
- import scikit-learn and run test models
'''